## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,78.85,83,95,10.36,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.91,71,75,27.63,broken clouds
2,2,Bontang,ID,0.1333,117.5000,83.30,69,98,10.25,heavy intensity rain
3,3,New Norfolk,AU,-42.7826,147.0587,78.66,36,73,3.00,broken clouds
4,4,Muisne,EC,0.6000,-80.0333,75.49,86,97,0.96,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,78.85,83,95,10.36,overcast clouds
2,2,Bontang,ID,0.1333,117.5000,83.30,69,98,10.25,heavy intensity rain
3,3,New Norfolk,AU,-42.7826,147.0587,78.66,36,73,3.00,broken clouds
4,4,Muisne,EC,0.6000,-80.0333,75.49,86,97,0.96,light rain
5,5,Vaini,TO,-21.2000,-175.2000,78.96,100,100,5.75,overcast clouds
12,12,Boa Vista,BR,2.8197,-60.6733,78.78,78,0,0.00,clear sky
13,13,Rikitea,PF,-23.1203,-134.9692,78.91,75,99,14.97,overcast clouds
18,18,Allapalli,IN,19.4167,80.0667,83.73,35,3,3.27,clear sky
22,22,Butaritari,KI,3.0707,172.7902,82.67,76,97,18.21,light rain
36,36,Bubaque,GW,11.2833,-15.8333,76.08,72,10,10.69,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,
2,Bontang,ID,83.30,heavy intensity rain,0.1333,117.5000,
3,New Norfolk,AU,78.66,broken clouds,-42.7826,147.0587,
4,Muisne,EC,75.49,light rain,0.6000,-80.0333,
5,Vaini,TO,78.96,overcast clouds,-21.2000,-175.2000,
12,Boa Vista,BR,78.78,clear sky,2.8197,-60.6733,
13,Rikitea,PF,78.91,overcast clouds,-23.1203,-134.9692,
18,Allapalli,IN,83.73,clear sky,19.4167,80.0667,
22,Butaritari,KI,82.67,light rain,3.0707,172.7902,
36,Bubaque,GW,76.08,clear sky,11.2833,-15.8333,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,78.85,overcast clouds,-21.2078,-159.7750,Paradise Inn
2,Bontang,ID,83.30,heavy intensity rain,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
3,New Norfolk,AU,78.66,broken clouds,-42.7826,147.0587,The Shingles Riverside Cottages
4,Muisne,EC,75.49,light rain,0.6000,-80.0333,Quinta Samay
5,Vaini,TO,78.96,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
663,Oranjestad,AW,79.32,clear sky,12.5240,-70.0270,Divi Village Golf & Beach Resort
664,Bambanglipuro,ID,87.93,overcast clouds,-7.9500,110.2833,Puri Brata
674,Vanimo,PG,84.31,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
677,Pemba,MZ,77.23,overcast clouds,-12.9740,40.5178,Guesthouse Samotina


In [21]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))